In [2]:
import pandas as pd
import numpy as np


In [3]:

df = pd.read_csv('../data/sell-in.txt', sep='\t')
df_productos_predecir = pd.read_csv('../data/product_id_apredecir201912.txt', sep='\t')

In [4]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [5]:
#Filter df to contain only products that are in df_productos_predecir
product_ids_to_predict = df_productos_predecir['product_id'].unique()
df = df[df['product_id'].isin(product_ids_to_predict)]

print(f"Original df shape after filtering: {df.shape}")
print(f"Unique products in df_productos_predecir: {len(product_ids_to_predict)}")
print(f"Unique products in filtered df: {df['product_id'].nunique()}")

Original df shape after filtering: (2293481, 7)
Unique products in df_productos_predecir: 780
Unique products in filtered df: 780


In [6]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [7]:
pivot_df = df.pivot_table(
    index=['customer_id', 'product_id'],
    columns='periodo',
    values='tn',
    aggfunc='sum'
).reset_index()

In [8]:
pivot_df.head()

periodo,customer_id,product_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,10001,20001,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,10001,20002,87.64856,66.08396,75.09182,49.51494,122.40283,167.46470,156.15120,18.15133,...,220.19153,155.81927,264.55349,151.12081,103.12062,148.91108,213.36148,430.90803,547.87849,334.03714
2,10001,20003,100.21284,126.97776,114.52896,37.34640,76.66386,108.30456,87.14160,43.57080,...,125.49948,86.54509,74.71874,78.79703,105.81480,121.06458,101.61982,196.18531,135.69192,137.98717
3,10001,20004,21.73954,29.76246,42.54996,9.31694,8.33349,10.92153,15.01063,12.42259,...,25.94769,17.84712,27.99741,34.26047,16.04585,8.33349,20.57492,37.88891,27.58851,12.94020
4,10001,20005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.66966,1.72238,4.25654,3.20851,5.41195,2.51269,5.66966,7.98907,11.01719,7.66693


In [9]:
# Fill NaN values following the rule: keep NaN for values before the first non-null value in each row
def fill_nans_after_first_value(row):
    # Get the time series columns (excluding customer_id and product_id)
    time_columns = row.index[2:]  # Assuming first 2 columns are customer_id and product_id
    
    # Find the first non-null index
    first_non_null_idx = None
    for idx in time_columns:
        if pd.notna(row[idx]):
            first_non_null_idx = idx
            break
    
    # If no non-null value found, return the row as is
    if first_non_null_idx is None:
        return row
    
    # Fill NaN values with 0 only after the first non-null value
    first_non_null_position = time_columns.get_loc(first_non_null_idx)
    for i in range(first_non_null_position + 1, len(time_columns)):
        col = time_columns[i]
        if pd.isna(row[col]):
            row[col] = 0
    
    return row

# Apply the function to fill NaN values
pivot_df_filled = pivot_df.apply(fill_nans_after_first_value, axis=1)

In [10]:
pivot_df_filled.head()

periodo,customer_id,product_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,10001.0,20001.0,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,10001.0,20002.0,87.64856,66.08396,75.09182,49.51494,122.40283,167.46470,156.15120,18.15133,...,220.19153,155.81927,264.55349,151.12081,103.12062,148.91108,213.36148,430.90803,547.87849,334.03714
2,10001.0,20003.0,100.21284,126.97776,114.52896,37.34640,76.66386,108.30456,87.14160,43.57080,...,125.49948,86.54509,74.71874,78.79703,105.81480,121.06458,101.61982,196.18531,135.69192,137.98717
3,10001.0,20004.0,21.73954,29.76246,42.54996,9.31694,8.33349,10.92153,15.01063,12.42259,...,25.94769,17.84712,27.99741,34.26047,16.04585,8.33349,20.57492,37.88891,27.58851,12.94020
4,10001.0,20005.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.66966,1.72238,4.25654,3.20851,5.41195,2.51269,5.66966,7.98907,11.01719,7.66693


In [11]:
# # Group by product_id and sum the values for each period, removing customer_id column
# product_summary = pivot_df_filled.drop('customer_id', axis=1).groupby('product_id').sum().reset_index()
# product_summary.head()

In [15]:
# Calculate percentage difference from average of last 3 months to 201810 vs 201812 and apply to predict 201912 from 201910
def calculate_growth_prediction(df):
    """
    Calculate percentage growth from average of 201808-201810 to 201812 and apply to 201910 to predict 201912
    Uses full granularity with product_id and customer_id
    Growth coefficient is calculated from 12 months average (dividing sum by count of non-null values)
    """
    results = []
    
    for idx, row in df.iterrows():
        product_id = row['product_id']
        customer_id = row['customer_id']
        
        # Get 12 months of data before 201812 (201801 to 201812)
        months_12 = [201801, 201802, 201803, 201804, 201805, 201806, 
                     201807, 201808, 201809, 201810, 201811, 201812]
        
        # Calculate average of 12 months (sum divided by count of non-null values)
        month_values = [row[month] for month in months_12 if pd.notna(row[month])]
        if len(month_values) > 0:
            avg_12_months = sum(month_values) / len(month_values)
        else:
            avg_12_months = 0
        
        # Calculate average of last 3 months before 201812
        avg_last_3_months = (row[201808] + row[201809] + row[201810]) / 3
        value_201812 = row[201812]
        value_201910 = row[201910]
        actual_201912 = row[201912]
        
        # Calculate growth coefficient from 12-month average to 201812
        if avg_12_months > 0:
            growth_coef = value_201812 / avg_12_months
        else:
            growth_coef = 1.0  # No growth if base value is 0
        
        # Apply growth coefficient to predict 201912 from 201910
        predicted_201912 = value_201910 * growth_coef
        
        results.append({
            'product_id': product_id,
            'customer_id': customer_id,
            'avg_12_months': avg_12_months,
            'non_null_months_count': len(month_values),
            'avg_201808_201810': avg_last_3_months,
            'value_201812': value_201812,
            'value_201910': value_201910,
            'actual_201912': actual_201912,
            'growth_coef': growth_coef,
            'predicted_201912': predicted_201912,
            'percentage_diff_12m_1812': ((value_201812 - avg_12_months) / avg_12_months * 100) if avg_12_months > 0 else 0
        })
    
    return pd.DataFrame(results)

# Apply the growth prediction method using full granularity
growth_predictions_df = calculate_growth_prediction(pivot_df_filled)
print("Growth-based predictions (using 12-month average with full granularity):")
print(growth_predictions_df.head(10))

# Calculate error metrics
growth_predictions_df['error_rate'] = abs((growth_predictions_df['predicted_201912'] - growth_predictions_df['actual_201912']) / growth_predictions_df['actual_201912']) * 100

print(f"\nMean Error Rate (Growth Method with 12-month avg): {growth_predictions_df['error_rate'].mean():.2f}%")
print(f"Median Error Rate (Growth Method with 12-month avg): {growth_predictions_df['error_rate'].median():.2f}%")
print(f"Total records processed: {len(growth_predictions_df)}")

Growth-based predictions (using 12-month average with full granularity):
   product_id  customer_id  avg_12_months  non_null_months_count  \
0     20001.0      10001.0     186.532245                     12   
1     20002.0      10001.0     202.363981                     12   
2     20003.0      10001.0      99.069007                     12   
3     20004.0      10001.0      20.773982                     12   
4     20005.0      10001.0       1.389497                      6   
5     20006.0      10001.0      17.307129                     12   
6     20007.0      10001.0      53.051688                     12   
7     20008.0      10001.0      21.675082                     12   
8     20009.0      10001.0      53.094397                     12   
9     20010.0      10001.0       0.449176                     12   

   avg_201808_201810  value_201812  value_201910  actual_201912  growth_coef  \
0         247.678430     254.62373     176.02980      180.21938     1.365039   
1         190.6317

In [16]:
growth_predictions_df.head(10)

,product_id,customer_id,avg_12_months,non_null_months_count,avg_201808_201810,value_201812,value_201910,actual_201912,growth_coef,predicted_201912,percentage_diff_12m_1812,error_rate
0,20001.0,10001.0,186.532245,12,247.678430,254.62373,176.02980,180.21938,1.365039,240.287486,36.503868,33.330548
1,20002.0,10001.0,202.363981,12,190.631780,287.14182,430.90803,334.03714,1.418937,611.431518,41.893740,83.042975
2,20003.0,10001.0,99.069007,12,100.059403,37.34640,196.18531,137.98717,0.376974,73.956682,-62.302640,46.403219
3,20004.0,10001.0,20.773982,12,26.940633,13.97542,37.88891,12.94020,0.672737,25.489261,-32.726329,96.977330
4,20005.0,10001.0,1.389497,6,0.708707,1.99727,7.98907,7.66693,1.437405,11.483532,43.740539,49.780059
5,20006.0,10001.0,17.307129,12,20.232167,9.44168,4.07780,15.84069,0.545537,2.224591,-45.446296,85.956477
6,20007.0,10001.0,53.051688,12,56.377237,36.54481,63.39886,55.22591,0.688853,43.672489,-31.114709,20.920290
7,20008.0,10001.0,21.675082,12,24.177527,10.97870,4.70516,12.70392,0.506513,2.383222,-49.348749,81.240260
8,20009.0,10001.0,53.094397,12,78.650647,61.57051,45.47683,74.67156,1.159642,52.736857,15.964233,29.374909
9,20010.0,10001.0,0.449176,12,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,-100.000000,NaN


In [13]:
growth_predictions_df.to_csv('../data/growth_predictions_prod_cust_3_month_avg.csv', index=False)

In [14]:
# # Calculate the total tonnage growth coefficient across all products
# total_201810 = product_summary[201810].sum()
# total_201812 = product_summary[201812].sum()

# # Calculate the overall growth coefficient
# overall_growth_coef = total_201812 / total_201810 if total_201810 > 0 else 1.0

# print(f"Total tonnage 201810: {total_201810:.2f}")
# print(f"Total tonnage 201812: {total_201812:.2f}")
# print(f"Overall growth coefficient: {overall_growth_coef:.4f}")
# print(f"Overall percentage change: {((total_201812 - total_201810) / total_201810 * 100):.2f}%")

# # Apply the reinforcement: multiply predicted_201912 by the overall growth coefficient
# growth_predictions_df['predicted_201912_reinforced'] = growth_predictions_df['predicted_201912'] * overall_growth_coef

# # Calculate error metrics for the reinforced predictions
# growth_predictions_df['error_rate_reinforced'] = abs((growth_predictions_df['predicted_201912_reinforced'] - growth_predictions_df['actual_201912']) / growth_predictions_df['actual_201912']) * 100

# print("\nComparison of predictions:")
# print("Growth-based predictions with reinforcement:")
# print(growth_predictions_df[['product_id', 'predicted_201912', 'predicted_201912_reinforced', 'actual_201912', 'error_rate', 'error_rate_reinforced']].head(10))

# print(f"\nOriginal Mean Error Rate: {growth_predictions_df['error_rate'].mean():.2f}%")
# print(f"Reinforced Mean Error Rate: {growth_predictions_df['error_rate_reinforced'].mean():.2f}%")
# print(f"Original Median Error Rate: {growth_predictions_df['error_rate'].median():.2f}%")
# print(f"Reinforced Median Error Rate: {growth_predictions_df['error_rate_reinforced'].median():.2f}%")
